In [1]:
from gensim.models import Word2Vec

In [32]:
corpus = './conversation.txt'
with open(corpus) as f:
    content = f.read()
parts = content.split("+++$+++")
parts = [part.strip().split('\n') for part in parts]
sentences = []
for part in parts:
    for i in range(len(part)):
        sentences.append("".join(part[i].split()))

model = Word2Vec(sentences, size=128, window=5, min_count= 6, workers=8)

In [33]:
for i in range(50):
    print(model.wv.index2word[i])

我
的
你
,
是
不
了
?
們
這
一
.
他
在
有
麼
個
好
。
人
!
來
要
那
會
就
什
到
說
沒
嗎
上
想
去
道
很
看
能
可
知
得
吧
以
過
對
她
都
爲
生
子


In [56]:
class Voc:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2:"PAD"}
        self.n_words = 3  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

def readVocs(corpus, corpus_name):
    print("Reading lines...")

    # combine every two lines into pairs and normalize
    with open(corpus) as f:
        content = f.read()
    parts = content.split("+++$+++")
    parts = [part.strip().split('\n') for part in parts]
    pairs = []
    for part in parts:
        for i in range(len(part) -1):
            pairs.append(["".join(part[i].split()), "".join(part[i+1].split())])

    voc = Voc(corpus_name)
    return voc, pairs

def filterPair(p):
    # input sequences need to preserve the last word for EOS_token
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def prepareData(corpus, corpus_name, word_model):
    voc, pairs = readVocs(corpus, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")

    for i in range(len(word_model.wv.vocab)):
        voc.addWord(word_model.wv.index2word[i])
    print("Counted words:", voc.n_words)
    directory = os.path.join(save_dir, 'training_data', corpus_name)
    if not os.path.exists(directory):
        os.makedirs(directory)
    torch.save(voc, os.path.join(directory, '{!s}.tar'.format('voc')))
    torch.save(pairs, os.path.join(directory, '{!s}.tar'.format('pairs')))
    return voc, pairs

def loadPrepareData(corpus):
    corpus_name = corpus.split('/')[-1].split('.')[0]
    try:
        print("Start loading training data ...")
        voc = torch.load(os.path.join(save_dir, 'training_data', corpus_name, 'voc.tar'))
        pairs = torch.load(os.path.join(save_dir, 'training_data', corpus_name, 'pairs.tar'))
    except FileNotFoundError:
        print("Saved data not found, start preparing trianing data ...")
        voc, pairs = prepareData(corpus, corpus_name)
    return voc, pairs

In [28]:
voc = Voc('test')

In [30]:
model.wv.index2word[1]

IndexError: list index out of range

In [63]:
MAX_LENGTH=10
voc, pairs = prepareData("./conversation.txt", 'movie_chat500W', model)

Reading lines...
Read 2729578 sentence pairs
Trimmed to 2729578 sentence pairs
Counting words...
Counted words: 5142
